In [ ]:
# matplotlib設定
import logging
import matplotlib

# フォント警告抑制
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

# DejaVu Sansフォント設定
matplotlib.rcParams['font.family'] = 'DejaVu Sans'
matplotlib.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Ubuntu']

import matplotlib.pyplot as plt
%matplotlib inline

print("✅ matplotlib設定完了")

# MLOps実験実行ノートブック

config駆動のMLOpsパイプライン実験を実行します。

## セル構成
1. **Import** - 必要なライブラリとモジュールのインポート
2. **データ読み込み・分割** - CSVデータの読み込みとtrain/test分割
3. **MLflow実行** - パイプライン構築、学習、評価、記録
4. **予測結果の確認と活用** - 予測結果DataFrameの確認と後続分析

## 1. Import

In [2]:
# 基本ライブラリ
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
import mlflow
from omegaconf import OmegaConf
import warnings
warnings.filterwarnings('ignore')

# データ処理ユーティリティ
from src.utils.data_utils import get_dataset_name, detect_task_type, load_csv_data
from src.utils.cv_utils import create_cv_strategy

# components機能インポート
from src.mlops.components.pipeline import create_pipeline
from src.mlops.components.visualization import create_visualizations
from src.mlops.components.optimization import OptunaOptimizer
from src.mlops.components.artifacts import (
    save_model_artifacts, log_experiment_metrics, 
    setup_mlflow_experiment, set_mlflow_tags, 
    log_config_parameters, log_runtime_parameters,
    create_prediction_dataframe, save_prediction_results
)

print("✅ ライブラリインポート完了")

✅ ライブラリインポート完了


## 2. Config読み込み・データ分割

In [ ]:
# Config読み込み（Hydraの代わりにOmegaConfで直接読み込み）
cfg = OmegaConf.load("config/config.yaml")

# pipelines設定を読み込んでマージ
pipeline_config = OmegaConf.load(f"config/pipelines/{cfg.defaults[2].pipelines}.yaml")
cfg = OmegaConf.merge(cfg, pipeline_config)

# models設定を読み込んでマージ
model_config = OmegaConf.load(f"config/models/classification/{cfg.defaults[1]['models/classification']}.yaml")
cfg = OmegaConf.merge(cfg, model_config)

print("📋 Config読み込み完了")
print(f"  - Pipeline: {cfg.defaults[2].pipelines}")
print(f"  - Model: {cfg.defaults[1]['models/classification']}")
print(f"  - CV Strategy: {cfg.evaluation.cv_strategy}")
print(f"  - Optuna: {'有効' if cfg.optuna.enabled else '無効'}")

# 注意: サンプリングはパイプライン内で実行されます（CVリーク防止のため）
# config/pipelines/universal_features.yamlでenabledをtrueに設定することで有効化

In [4]:
# データ読み込み
print("📊 データ読み込み開始")
df, feature_cols, target_names = load_csv_data(cfg)

# データ分割
X = df[feature_cols]
y = df[cfg.data.target_column]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=cfg.data.test_size,
    random_state=cfg.data.random_state
)

# タスクタイプ判定
task_type = detect_task_type(y)

print(f"✅ データ準備完了")
print(f"  - データセット: {cfg.data.file_path}")
print(f"  - データ形状: {df.shape}")
print(f"  - 特徴量数: {len(feature_cols)}")
print(f"  - タスクタイプ: {task_type}")
print(f"  - Train/Test: {len(X_train)}/{len(X_test)}")
print(f"  - クラス数: {len(target_names) if target_names is not None else 'N/A'}")

📊 データ読み込み開始
📊 CSV: _data/raw/wine_classification.csv - Shape: (178, 15) - Features: 13 - Classes: 3
✅ データ準備完了
  - データセット: _data/raw/wine_classification.csv
  - データ形状: (178, 15)
  - 特徴量数: 13
  - タスクタイプ: classification
  - Train/Test: 142/36
  - クラス数: 3


## 3. MLflow実験実行

In [ ]:
    cv_strategy = create_cv_strategy(cfg)
    print(f"  - CV戦略: {cfg.evaluation.cv_strategy['class']} (n_splits={cfg.evaluation.cv_strategy.params.n_splits})")

## 4. 予測結果の確認と活用

予測結果DataFrameを確認し、後続の分析に活用できます。

In [6]:
# 予測結果DataFrameの確認
if df_predictions is not None:
    print("📊 予測結果DataFrame情報")
    print(f"  - データ件数: {len(df_predictions)}件")
    print(f"  - カラム数: {len(df_predictions.columns)}列")
    
    # 分類タスクの場合のカラム情報
    if task_type == "classification":
        proba_cols = [col for col in df_predictions.columns if col.startswith('proba_class_')]
        print(f"  - 確率カラム: {proba_cols}")
        print(f"  - 信頼度カラム: prediction_confidence")
    
    print("\n📋 予測結果サンプル（先頭5件）:")
    display(df_predictions.head())
else:
    print("⚠️ 予測結果DataFrameがまだ作成されていません。上のセルを実行してください。")

📊 予測結果DataFrame情報
  - データ件数: 36件
  - カラム数: 29列
  - 確率カラム: ['proba_class_0', 'proba_class_1', 'proba_class_2']
  - 信頼度カラム: prediction_confidence

📋 予測結果サンプル（先頭5件）:


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,...,generated_feature_6,generated_feature_7,generated_feature_8,generated_feature_9,y_true,y_pred,proba_class_0,proba_class_1,proba_class_2,prediction_confidence
19,13.64,3.10,2.56,15.2,116.0,2.70,3.03,0.17,1.66,5.10,...,0.018252,0.015517,1.066134,0.365487,0,0,0.999012,0.000603,0.000385,0.999012
45,14.21,4.04,2.44,18.9,111.0,2.85,2.65,0.30,1.25,5.24,...,0.078527,-0.370294,1.024444,1.145552,0,0,0.998883,0.000613,0.000504,0.998883
140,12.93,2.81,2.70,21.0,96.0,1.54,0.50,0.53,0.75,4.60,...,-0.197015,-0.798972,-0.393023,-0.447771,2,2,0.002974,0.012048,0.984978,0.984978
30,13.73,1.50,2.70,22.5,101.0,3.00,3.25,0.29,2.38,5.70,...,0.276573,1.001477,0.162846,1.826033,0,0,0.994862,0.003989,0.001149,0.994862
67,12.37,1.17,1.92,19.6,78.0,2.11,2.00,0.27,1.04,4.68,...,-0.162572,0.701402,1.232895,-0.746519,1,1,0.003374,0.993641,0.002985,0.993641


In [7]:
# 予測精度の詳細分析（分類タスクの例）
if df_predictions is not None and task_type == "classification":
    # 予測の信頼度分布
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 4))
    
    # 信頼度ヒストグラム
    plt.subplot(1, 2, 1)
    plt.hist(df_predictions['prediction_confidence'], bins=20, edgecolor='black')
    plt.xlabel('予測信頼度')
    plt.ylabel('件数')
    plt.title('予測信頼度の分布')
    
    # 正誤別の信頼度
    plt.subplot(1, 2, 2)
    df_predictions['is_correct'] = df_predictions['y_true'] == df_predictions['y_pred']
    correct_conf = df_predictions[df_predictions['is_correct']]['prediction_confidence']
    incorrect_conf = df_predictions[~df_predictions['is_correct']]['prediction_confidence']
    
    plt.boxplot([correct_conf, incorrect_conf], labels=['正解', '不正解'])
    plt.ylabel('予測信頼度')
    plt.title('正誤別の予測信頼度')
    
    plt.tight_layout()
    plt.show()
    
    # 統計情報
    print("📊 予測信頼度の統計:")
    print(f"  - 全体平均: {df_predictions['prediction_confidence'].mean():.3f}")
    print(f"  - 正解時平均: {correct_conf.mean():.3f}")
    print(f"  - 不正解時平均: {incorrect_conf.mean():.3f}")

findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Liberation Sans, Bitstream Vera Sans, sans-serif
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Liberation Sans, Bitstream Vera Sans, sans-serif
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Liberation Sans, Bitstream Vera Sans, sans-serif
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Liberation Sans, Bitstream Vera Sans, sans-serif
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Liberation Sans, Bitstream Vera Sans, sans-serif
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Liberation Sans, Bitstream Vera Sans, sans-serif
findfont: Generic family 'sans-serif' not found because none of the fo

📊 予測信頼度の統計:
  - 全体平均: 0.975
  - 正解時平均: 0.985
  - 不正解時平均: 0.627


## 追加: データ分析とエクスポート

予測結果DataFrameはそのまま後続の分析に利用可能です。

In [8]:
# 予測結果をローカルCSVとして保存（必要な場合）
if df_predictions is not None:
    output_path = "test_predictions_local.csv"
    df_predictions.to_csv(output_path, index=False)
    print(f"✅ 予測結果をローカル保存: {output_path}")
    
    # 必要なカラムのみ抽出した例
    df_essential = df_predictions[['y_true', 'y_pred', 'prediction_confidence']]
    print(f"\n📋 エッセンシャル予測結果（y_true, y_pred, confidence）:")
    display(df_essential.head())

✅ 予測結果をローカル保存: test_predictions_local.csv

📋 エッセンシャル予測結果（y_true, y_pred, confidence）:


,y_true,y_pred,prediction_confidence
19,0,0,0.999012
45,0,0,0.998883
140,2,2,0.984978
30,0,0,0.994862
67,1,1,0.993641


## MLflow UIの起動

実験結果を確認するには、ターミナルで以下を実行：
```bash
mlflow ui
```

その後、ブラウザで http://localhost:5000 にアクセス